# Summary
The purpose of this code is to create a TensorFlow Lite model. To do this, we take in a data set, create a TensorFlow model, train the model on that data set, then convert the TF model into a TF Lite model. This TF Lite model can then be loaded onto a mobile device for audio classification. 

There are a variety of models that can be trained on a variety of data sets. This code is for training amplitude models, which are the most straight foraward. This guide aims to describe all of the parameters in the training process, so that you can change them and build your own models. Remember, the end goal is to acheive the highest validation accuracy possible before loading that model onto a phone for use.

**Numbers to beat for our l6-data:** <br>
*A good goal--* 85% val_acc <br>
*Our best model--* 91% val_acc

# Imports
We use Keras (which is built on top of Tensorflow) to build and train our models. Librosa is used for audio processing.

In [1]:
# make sure kernel matches pip version
!pip3 install -r requirements.txt

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Tensorflow
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

# Keras
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import (Activation, Dense, Dropout, Flatten, Conv2D, Conv1D, 
                          MaxPooling2D, GlobalAveragePooling2D, MaxPooling1D, Lambda)
from keras.layers.normalization import BatchNormalization
from keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint
from keras.utils import to_categorical, multi_gpu_model
import keras.backend as K

import librosa
import multiprocessing as mp
import numpy as np
import scipy.io.wavfile
from scipy.fftpack import dct
from sklearn.model_selection import train_test_split
from tqdm import tqdm as tqdm
import time
from pprint import pprint
import uuid
import glob
import math

Using TensorFlow backend.


# Constants

## General

**RAW_DATA_DIR:** Where the raw training data is located. In the directory each folder name is a label, and its contents are .wav files corresponding to that label. See the "l6-data" directory for an example. 

**AMP_PROCESSED_DATA_DIR:** Where the processed training data is to be stored. After processing the data, it will be populated with numpy files. Each numpy file is named after a label, and contains all of the training data for that label stored as a 3d numpy array. More info on this later.

**AUDIO_LENGTH:** The desired input size for the model. An input size of 44100 at 44100 Hz would be a one second input. 

**SAMPLE_RATE:** The sample rate of the microphone.

## Training

**channel:** We use a one channel audio input

**epochs:** This is how many times the model will train on your data. 200 is usually a good number, there are diminishing returns after a certain point. 

**batch_size:** I don't even know what this is.

**verbose:** 1 is true, 0 is false. Its always a good idea to have this on. 

**num_classes:** The number of classes (also referred to as labels) in your data set.

In [2]:
RAW_DATA_DIR = "data/"
AMP_PROCESSED_DATA_DIR = "amp-processed-data/"
MODEL_NAME = 'Audio_Recorder'
AUDIO_LENGTH = 44100
SAMPLE_RATE = 44100

channel = 1
epochs = 1
batch_size = 128
verbose = 1
num_classes = 6

# Data Processing

### get_labels(path)
**Input:** `RAW_DATA_DIR` <br>
**Output:** `Tuple (Labels, Indices of the labels, one-hot encoded labels)` <br>
**Description:** Gets all labels (aka filenames) inside your raw data directory. The indices are the positions of these labels, and the hot encoded vector is a vector of zeros the length of the number of labels, with a 1 at the corresponding label's index. More info can be found about these online.

### label_to_amplitude_vecs(args)
**Input:** `Tuple (label, input_path, output_path, tqdm_position)`<br>
**Output:** Shape of the generated numpy file (stored under `output_path`)<br>
**Description:** This function is called by `process_data_amplitude()` in parallel to convert a label's raw .wav files to a single numpy file. This can take a while, so David made a super fancy tdqm display (hence the tqdm_position parameter). **This function will split the training data into overlapping arrays the size of input length, then store them in a 3d numpy array under the `output_path` dir.** It will move though the clip at a rate of AUDIO_LENGTH / 2, so if a clip is 10.1 seconds long, and the input length is 44100 with a 44100 Hz sampling rate, it will be processed as 21 one second clips. The final piece of the clip is padded with zeros to match the input length. This is done for all the .wav files in a label, which are converted to amplitude arrays with `librosa.load()`. The resulting numpy array contains the numbers that are actually trained on.

### process_data_amplitude(input_path, output_path)
**Input:** RAW_DATA_DIR, AMP_PROCESSED_DATA_DIR<br>
**Output:** Shape of the numpy files stored in output_path<br>
**Description:** Calls ` label_to_amplitude_vecs()`, and populates output_path with numpy files. Each numpy file is named after a label, and its content is a 3d numpy array with all the training data for that label. The shape of this array is (number_of_files, AUDIO_LENGTH, 1). The 3rd empty dimension is required by Keras.

In [3]:
def get_labels(path):
    labels = [i for i in sorted(os.listdir(path)) if i[0] != "."]
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [4]:
def label_to_amplitude_vecs(args) -> None:
    label, input_path, output_path, tqdm_position = args

    # Get all audio files for this label
    wavfiles = [os.path.join(input_path, label, wavfile) for wavfile in os.listdir(os.path.join(input_path, label))]

    # tqdm is amazing, so print all the things this way
    print(" ", end="", flush=True)
    twavs = tqdm(wavfiles, position=tqdm_position)
    
    vectors = []
    for i, wavfile in enumerate(twavs):
        # Load the audio file; this also works for .flac files
        audio_buf, _ = librosa.load(wavfile, mono=True, sr=SAMPLE_RATE)
        audio_buf = audio_buf.reshape(-1, 1)
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)            
        remaining_buf = audio_buf.copy()
        while remaining_buf.shape[0] > AUDIO_LENGTH:
            # Add the first AUDIO_LENGTH of the buffer as a new vector to train on
            new_buf = remaining_buf[ : AUDIO_LENGTH ]
            vectors.append(new_buf)
            
            # Remove 1/2 * AUDIO_LENGTH from the front of the buffer
            remaining_buf = remaining_buf[ int(AUDIO_LENGTH / 2) : ]
            
        # Whatever is left, pad and stick in the training data
        remaining_buf = np.concatenate((remaining_buf, np.zeros(shape=(AUDIO_LENGTH - len(remaining_buf), 1))))
        vectors.append(remaining_buf)
        
        # Update tqdm
        twavs.set_description("Label - '{}'".format(label))
        twavs.refresh()
    np_vectors = np.array(vectors)
    np.save(os.path.join(output_path, label + '.npy'), np_vectors)
    return np_vectors.shape
    
def process_data_amplitude(input_path, output_path):
    
    labels, _, _ = get_labels(input_path)
    pool = mp.Pool()
    result = pool.map(label_to_amplitude_vecs, 
                     [(label, input_path, output_path, tqdm_position) 
                          for tqdm_position, label in enumerate(labels)])
    pool.close()
    return result

In [5]:
!ls amp-processed-data

In [6]:
process_data_amplitude(RAW_DATA_DIR, AMP_PROCESSED_DATA_DIR)


  0%|          | 0/202 [00:00<?, ?it/s]


  0%|          | 0/146 [00:00<?, ?it/s]

  0%|          | 0/169 [00:00<?, ?it/s]



  0%|          | 0/407 [00:00<?, ?it/s]




Label - 'Coughing':   0%|          | 0/286 [00:00<?, ?it/s]

Label - 'Finger_snapping':   0%|          | 0/169 [00:00<?, ?it/s]

Label - 'Finger_snapping':   0%|          | 0/169 [00:00<?, ?it/s]



Label - 'Coughing':   0%|          | 0/286 [00:00<?, ?it/s]



Label - 'Knock':   0%|          | 0/407 [00:00<?, ?it/s]




Label - 'Coughing':   0%|          | 0/286 [00:00<?, ?it/s]

Label - 'Finger_snapping':   0%|          | 0/169 [00:00<?, ?it/s]


Label - 'Computer_keyboard':   0%|          | 0/202 [00:00<?, ?it/s][A




Label - 'Coughing':   1%|          | 2/286 [00:00<00:19, 14.40it/s]

Label - 'Finger_snapping':   0%|          | 0/169 [00:00<?, ?it/s]



Label - 'Knock':   0%|          | 0/407 [00:00<?, ?it/s]


Label - 'Computer_keyboard':   0%|          | 0/202 [00:00<?, ?it/s][A




Label - 'Laughing':   0%|   

Label - 'Coughing':   4%|▍         | 11/286 [00:01<00:25, 10.99it/s]


Label - 'Keys_jangling':   5%|▍         | 7/146 [00:01<00:17,  7.79it/s]



Label - 'Computer_keyboard':   3%|▎         | 7/202 [00:01<00:27,  7.20it/s]


Label - 'Coughing':   4%|▍         | 11/286 [00:01<00:25, 10.99it/s]



Label - 'Coughing':   4%|▍         | 11/286 [00:01<00:25, 10.99it/s]



Label - 'Computer_keyboard':   4%|▍         | 8/202 [00:01<00:25,  7.56it/s]A


Label - 'Keys_jangling':   5%|▍         | 7/146 [00:01<00:17,  7.79it/s]


Label - 'Coughing':   4%|▍         | 11/286 [00:01<00:25, 10.99it/s]


Label - 'Keys_jangling':   6%|▌         | 9/146 [00:01<00:15,  8.79it/s]



Label - 'Coughing':   4%|▍         | 11/286 [00:01<00:25, 10.99it/s]



Label - 'Computer_keyboard':   4%|▍         | 9/202 [00:01<00:23,  8.11it/s]



Label - 'Knock':   3%|▎         | 11/407 [00:01<00:38, 10.22it/s]



Label - 'Knock':   3%|▎         | 11/407 [00:01<00:38, 10.22it/s]


Label - 'Coughing':   5%|▍         | 13

Label - 'Computer_keyboard':   9%|▉         | 19/202 [00:01<00:14, 12.50it/s]


Label - 'Keys_jangling':  13%|█▎        | 19/146 [00:01<00:10, 12.23it/s]

Label - 'Coughing':   8%|▊         | 23/286 [00:02<00:19, 13.58it/s]


Label - 'Keys_jangling':  13%|█▎        | 19/146 [00:02<00:10, 12.23it/s]

Label - 'Coughing':   8%|▊         | 23/286 [00:02<00:19, 13.58it/s]


Label - 'Computer_keyboard':   9%|▉         | 19/202 [00:02<00:14, 12.50it/s][A

Label - 'Computer_keyboard':  10%|█         | 21/202 [00:02<00:13, 12.99it/s]

Label - 'Coughing':   8%|▊         | 23/286 [00:02<00:19, 13.58it/s]


Label - 'Coughing':   9%|▊         | 25/286 [00:02<00:19, 13.11it/s]

Label - 'Finger_snapping':   3%|▎         | 5/169 [00:02<01:30,  1.81it/s]


Label - 'Computer_keyboard':  10%|█         | 21/202 [00:02<00:13, 12.99it/s][A

Label - 'Coughing':   9%|▊         | 25/286 [00:02<00:19, 13.11it/s]

Label - 'Finger_snapping':   4%|▍         | 7/169 [00:02<01:06,  2.45it/s]


Label - 'Coughing':   

Label - 'Computer_keyboard':  15%|█▌        | 31/202 [00:02<00:14, 11.83it/s]




Label - 'Coughing':  12%|█▏        | 35/286 [00:02<00:20, 12.29it/s]

Label - 'Finger_snapping':   9%|▉         | 16/169 [00:02<00:20,  7.34it/s]


Label - 'Computer_keyboard':  15%|█▌        | 31/202 [00:02<00:14, 11.83it/s][A

Label - 'Finger_snapping':   9%|▉         | 16/169 [00:02<00:20,  7.34it/s]


Label - 'Computer_keyboard':  15%|█▌        | 31/202 [00:03<00:14, 11.83it/s][A

Label - 'Finger_snapping':   9%|▉         | 16/169 [00:03<00:20,  7.34it/s]

Label - 'Finger_snapping':   9%|▉         | 16/169 [00:03<00:20,  7.34it/s]

Label - 'Computer_keyboard':  16%|█▋        | 33/202 [00:03<00:13, 12.66it/s]


Label - 'Keys_jangling':  21%|██        | 31/146 [00:03<00:10, 11.46it/s]


Label - 'Keys_jangling':  21%|██        | 31/146 [00:03<00:10, 11.46it/s]

Label - 'Finger_snapping':  11%|█         | 18/169 [00:03<00:17,  8.67it/s]


Label - 'Coughing':  12%|█▏        | 35/286 [00:03<00:20, 12.29it/s

Label - 'Keys_jangling':  27%|██▋       | 39/146 [00:03<00:09, 11.57it/s]



Label - 'Computer_keyboard':  19%|█▉        | 39/202 [00:03<00:14, 11.09it/s]


Label - 'Keys_jangling':  27%|██▋       | 39/146 [00:03<00:09, 11.57it/s]

Label - 'Computer_keyboard':  19%|█▉        | 39/202 [00:03<00:14, 11.09it/s]


Label - 'Keys_jangling':  28%|██▊       | 41/146 [00:03<00:09, 11.12it/s]



Label - 'Knock':   7%|▋         | 29/407 [00:03<01:07,  5.61it/s]




Label - 'Laughing':   4%|▍         | 17/398 [00:03<00:57,  6.68it/s]

Label - 'Coughing':  15%|█▌        | 43/286 [00:03<00:23, 10.55it/s]



Label - 'Knock':   7%|▋         | 29/407 [00:03<01:07,  5.61it/s]


Label - 'Keys_jangling':  28%|██▊       | 41/146 [00:03<00:09, 11.12it/s]




Label - 'Coughing':  15%|█▌        | 43/286 [00:03<00:23, 10.55it/s]


Label - 'Keys_jangling':  28%|██▊       | 41/146 [00:03<00:09, 11.12it/s]

Label - 'Computer_keyboard':  20%|██        | 41/202 [00:03<00:14, 11.15it/s]A




Label - 'Laughing':   5%

...

Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  67%|██████▋   | 267/398 [00:45<00:33,  3.95it/s]




Label - 'Laughing':  68%|██████▊   | 271/398 [00:45<00:23,  5.35it/s]




Label - 'Laughing':  68%|██████▊   | 271/398 [00:45<00:23,  5.35it/s]




Label - 'Laughing':  68%|██████▊   | 271/398 [00:45<00:23,  5.35it/s]




Label - 'Laughing':  68%|██████▊   | 271/398 [00:45<00:23,  5.35it/s]




Label - 'Laughing':  68%|██████▊   | 2

Label - 'Laughing':  77%|███████▋  | 306/398 [00:46<00:03, 24.75it/s]




Label - 'Laughing':  77%|███████▋  | 306/398 [00:46<00:03, 24.75it/s]




Label - 'Laughing':  77%|███████▋  | 306/398 [00:46<00:03, 24.75it/s]




Label - 'Laughing':  77%|███████▋  | 306/398 [00:47<00:03, 24.75it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  78%|███████▊  | 309/398 [00:47<00:06, 14.65it/s]




Label - 'Laughing':  79%|███████▊  | 3

Label - 'Laughing':  87%|████████▋ | 347/398 [00:48<00:01, 27.16it/s]




Label - 'Laughing':  87%|████████▋ | 347/398 [00:48<00:01, 27.16it/s]




Label - 'Laughing':  87%|████████▋ | 347/398 [00:48<00:01, 27.16it/s]




Label - 'Laughing':  87%|████████▋ | 347/398 [00:48<00:01, 27.16it/s]




Label - 'Laughing':  87%|████████▋ | 347/398 [00:48<00:01, 27.16it/s]




Label - 'Laughing':  87%|████████▋ | 347/398 [00:48<00:01, 27.16it/s]




Label - 'Laughing':  87%|████████▋ | 347/398 [00:48<00:01, 27.16it/s]




Label - 'Laughing':  87%|████████▋ | 347/398 [00:48<00:01, 27.16it/s]




Label - 'Laughing':  88%|████████▊ | 351/398 [00:48<00:01, 28.55it/s]




Label - 'Laughing':  88%|████████▊ | 351/398 [00:48<00:01, 28.55it/s]




Label - 'Laughing':  88%|████████▊ | 351/398 [00:48<00:01, 28.55it/s]




Label - 'Laughing':  88%|████████▊ | 351/398 [00:48<00:01, 28.55it/s]




Label - 'Laughing':  88%|████████▊ | 351/398 [00:48<00:01, 28.55it/s]




Label - 'Laughing':  88%|████████▊ | 3

Label - 'Laughing':  97%|█████████▋| 386/398 [00:50<00:00, 28.92it/s]




Label - 'Laughing':  97%|█████████▋| 386/398 [00:50<00:00, 28.92it/s]




Label - 'Laughing':  97%|█████████▋| 386/398 [00:50<00:00, 28.92it/s]




Label - 'Laughing':  97%|█████████▋| 386/398 [00:50<00:00, 28.92it/s]




Label - 'Laughing':  97%|█████████▋| 386/398 [00:50<00:00, 28.92it/s]




Label - 'Laughing':  98%|█████████▊| 390/398 [00:50<00:00, 29.71it/s]




Label - 'Laughing':  98%|█████████▊| 390/398 [00:50<00:00, 29.71it/s]




Label - 'Laughing':  98%|█████████▊| 390/398 [00:50<00:00, 29.71it/s]




Label - 'Laughing':  98%|█████████▊| 390/398 [00:50<00:00, 29.71it/s]




Label - 'Laughing':  98%|█████████▊| 390/398 [00:50<00:00, 29.71it/s]




Label - 'Laughing':  98%|█████████▊| 390/398 [00:50<00:00, 29.71it/s]




Label - 'Laughing':  98%|█████████▊| 390/398 [00:50<00:00, 29.71it/s]




Label - 'Laughing':  98%|█████████▊| 390/398 [00:50<00:00, 29.71it/s]




Label - 'Laughing':  98%|█████████▊| 3

[(4255, 44100, 1),
 (2930, 44100, 1),
 (911, 44100, 1),
 (2204, 44100, 1),
 (3043, 44100, 1),
 (6057, 44100, 1)]

# Training the Model

### get_train_test(split_ratio: float)
**Inputs:** `split_ratio` <br>
**Outputs:** `X_train, X_test, y_train, y_test` <br>
**Description:** Uses a sklearn library to split the processed data into training data and test data. We use a .75 ratio of training to test data. X_train is the training data, which is a 3d array with dimnesions (number_of_clips, input_size, 1). Again, the 3rd dimension is a requirement from Keras. y_train is a 1d array containing label indices that correspond to each clip in X_train. X_test and y_test are the same, but for testing data instead of training data. The testing data is used to calculate accuracy metrics during training.
### get_model()
**Inputs:** None <br>
**Outputs:** `Tensorflow Model` <br> 
**Description:** This function is where the model itself is built. The input size must match the clip length (e.g. our input size variable), and the output must be a set of weights corresponding to each label (so an array of length labels). The highest weight will correspond to the final classification. The models we use are CNNs, which are built layer by layer. In general, a larger input size means that more layers can be used to learn on that data. Because the amplitude input is fairly large (44000 elements, compared to MFCC with just 11136), this model should be fairly large as well. Any model with the proper input and output sizes can be loaded. In our case, the input shape was (44100,1), as Keras required the extra dimension. 

### train(model, X_train, y_train_hot, X_test, y_test_hot)
**Inputs:** `model, X_train, y_train_hot, X_test, y_test_hot`<br>
**Outputs:** The given model is trained on the given data. <br>
**Description:** We use 'adam' optimization during training, as it usually results in much higher accuracies. We also save the model at its highest validation accuracies as it trains, so that we can retrieve the model from the epoch with the highest accuracy. These are saved as "weights-{val_acc}.hdf5", and can be converted to TF Lite models later.

In [ ]:
def get_train_test(split_ratio: float):
    
    assert split_ratio < 1 and split_ratio > 0
    
    # Get available labels
    labels, indices, _ = get_labels(RAW_DATA_DIR)

    # Getting first arrays
    X = np.load(AMP_PROCESSED_DATA_DIR + labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in tqdm(enumerate(labels[1:])):
        
        x = np.load(AMP_PROCESSED_DATA_DIR + label + '.npy')
        print(AMP_PROCESSED_DATA_DIR + label + '.npy')
        print (X.shape)
        print (x.shape)
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    # # Loading train set and test set
    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=42, shuffle=True)

X_train, X_test, y_train, y_test = get_train_test(0.75)
        
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print("Converting to categorical")
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train.shape)
print(y_test.shape)

In [ ]:
def get_model():
    m = Sequential()
    m.add(Conv1D(64,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001),
                 name='voice'))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))

    for i in range(2):
        m.add(Conv1D(64,
                     kernel_size=3,
                     strides=1,
                     padding='same',
                     kernel_initializer='glorot_uniform',
                     kernel_regularizer=regularizers.l2(l=0.0001)))
        m.add(BatchNormalization())
        m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))

    for i in range(2):
        m.add(Conv1D(128,
                     kernel_size=3,
                     strides=1,
                     padding='same',
                     kernel_initializer='glorot_uniform',
                     kernel_regularizer=regularizers.l2(l=0.0001)))
        m.add(BatchNormalization())
        m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))

    for i in range(3):
        m.add(Conv1D(256,
                     kernel_size=3,
                     strides=1,
                     padding='same',
                     kernel_initializer='glorot_uniform',
                     kernel_regularizer=regularizers.l2(l=0.0001)))
        m.add(BatchNormalization())
        m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))

    for i in range(2):
        m.add(Conv1D(512,
                     kernel_size=3,
                     strides=1,
                     padding='same',
                     kernel_initializer='glorot_uniform',
                     kernel_regularizer=regularizers.l2(l=0.0001)))
        m.add(BatchNormalization())
        m.add(Activation('relu'))

    m.add(Lambda(lambda x: K.mean(x, axis=1))) # Same as GAP for 1D Conv Layer
    m.add(Dense(num_classes, activation='softmax', name='label'))
    return m

In [ ]:
def train(model, X_train, y_train_hot, X_test, y_test_hot):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print(model.summary())
    
    reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
    mcp_save = ModelCheckpoint('out/weights-{val_acc:.2f}.hdf5', save_best_only=True, monitor='val_acc', mode='max')
    model.fit(X_train, 
              y_train_hot, 
              batch_size=batch_size, 
              epochs=epochs, 
              verbose=verbose, 
              validation_data=(X_test, y_test_hot),
              callbacks=[reduce_lr, mcp_save],
              shuffle=True)

In [ ]:
if not os.path.exists('out'):
    os.mkdir('out')
        
model = get_model()
train(model, X_train, y_train, X_test, y_test)
model.save("out/amplitude-l6.hdf5")

# Converting to TF Lite
**frozen_model_name:** The frozen model to convert. These are saved automatically in the `out/` directory during training.<br>
**tf_lite_model_name:** The desired name of your TF model. We follow the following format: `dataset_feature_'i'inputsize_'l'labelsize`. So, a model trained on 6 office sounds with 1 second of amplitude at a sample rate of 44100 Hz would be named `OfficeSounds_Amplitude_i44100_l6.tflite`.

In [ ]:
frozen_model_name = 'out/weights-0.91.hdf5'
tf_lite_model_name = "amplitude-l6-acc91.tflite"

converter = tf.lite.TFLiteConverter.from_keras_model_file(frozen_model)
tfmodel = converter.convert()
open (tf_lite_name, "wb") .write(tfmodel)

# Predicting
A possibly unfinished, definitely never used function to actually predict labels on the computer side. 

In [ ]:
def predict(filepath, model):
    """Predict the classification of a single audio sample"""
    audio_buf, _ = librosa.load(filepath, mono=True, sr=SAMPLE_RATE)
    audio_buf = audio_buf.reshape(-1, 1)
    audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)

    remaining_buf = audio_buf.copy()
    wave_vectors = []
    while remaining_buf.shape[0] > AUDIO_LENGTH:
        # Add the first AUDIO_LENGTH of the buffer as a new vector to train on
        new_buf = remaining_buf[ : AUDIO_LENGTH ]
        wave_vectors.append(new_buf)

        # Shrink the buffer by AUDIO_LENGTH
        remaining_buf = remaining_buf[ AUDIO_LENGTH : ]

    # Whatever is left, pad and stick in the training data
    remaining_buf = np.concatenate((remaining_buf, np.zeros(shape=(AUDIO_LENGTH - len(remaining_buf), 1))))
    wave_vectors.append(remaining_buf)
    
    # wave_vectors now contains every segment of the provided audio input. 
    # Get the classification for the sample by predicting on each segment, 
    # and then taking a majority vote.
    
    def most_common(lst):
        return max(set(lst), key=lst.count)
    
    def most_confident(lst):
        """Return the label with the highest confidence"""
        return 
    
    labels, _, _ = get_labels(DATA_PATH)
    predicted_labels = []
    predicted_confs = []
    
#     for i, vec in enumerate(wave_vectors):
#         inp = np.array([vec])
#         label_vec = model.predict(inp)
#         best_label_idx = np.argmax(label_vec, axis=1)[0]
#         predicted_confs.append(np.asscalar(np.max(label_vec, axis=1)))
#         predicted_labels.append(best_label_idx)
# #         print("label for {}: {}".format(i, best_label_idx))

    label_vec = model.predict(wave_vectors)
    best_label_idx = np.argmax(label_vec, axis=1)[0]
    predicted_confs.append(np.asscalar(np.max(label_vec, axis=1)))
    predicted_labels.append(best_label_idx)
#         print("label for {}: {}".format(i, best_label_idx))
        
    print(predicted_labels)
    print(predicted_confs)
    
    # returns the label recognized with the highest confidence
    best_total_label_idx = predicted_labels[np.argmax(predicted_confs)]
    
    return best_total_label_idx

In [ ]:
import glob
num_correct = 0
num_total = 0
# files = glob.glob("esc50/keyboard_typing/*.wav")
files = glob.glob("keyboard.wav")
for f in files:
    label_idx = predict(f, model)
    
    string_label = labels[label_idx]
    print("Predicted label: {}".format(string_label))
    
    if label_idx == 3:
        num_correct += 1
    num_total += 1
print("Number correct: {}".format(num_correct))
print("Total files: {}".format(num_total))